In [150]:
import numpy as np
from math import *
import os
from os.path import join
import bpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
from numpy.linalg import norm
import torch
import lie_learn.spaces.S2 as S2

In [141]:
def create_cube_grid():
    x_ = np.linspace(-1., 1., 4)
    y_ = np.linspace(-1., 1., 4)
    z_ = np.linspace(-1., 1., 4)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    
    return x, y, z

In [142]:
def create_cube_centers():
    x_ = np.linspace(-2/3, 2/3, 3)
    y_ = np.linspace(-2/3, 2/3, 3)
    z_ = np.linspace(-2/3, 2/3, 3)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    
    return x, y, z

In [143]:
x_axis, y_axis, z_axis = create_cube_grid()
x_axis_, y_axis_, z_axis_ = create_cube_centers()

In [156]:
fig = ipv.figure()
ipv.scatter(x_axis, y_axis, z_axis, marker="sphere", color="red")
ipv.scatter(x_axis_, y_axis_, z_axis_, marker="sphere", color="blue")
ipv.show()
ipv.save("plt.html")

AttributeError: module 'ipyvolume' has no attribute 'close'

In [145]:
centers = np.array(list(zip(x_axis_, y_axis_, z_axis_)))

In [146]:
fig = ipv.figure()
R = 2 * 1/3 * np.sqrt(3) * 1/2
for center in centers:
    x = R * np.outer(np.cos(u), np.sin(v)) + center[0]
    y = R * np.outer(np.sin(u), np.sin(v))  + center[1]
    z = R * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]
    ipv.plot_mesh(x, y, z, u=y, v=x, wireframe=False, color="red")
ipv.show()

In [182]:
def get_grids(b, num_grids, base_radius=1, center=[0, 0, 0], grid_type="Driscoll-Healy"):
    """
    :param b: the number of grids on the sphere
    :param base_radius: the radius of each sphere
    :param grid_type: "Driscoll-Healy"
    :param num_grids: number of grids
    :return: [(radius, tensor([2b, 2b, 3])) * num_grids]
    """

    grids = list()
    radiuses = [round(i, 2) for i in list(np.linspace(0, base_radius, num_grids + 1))[1:]]

    # Each grid has differet radius, the radiuses are distributed uniformly based on number
    for radius in radiuses:

        # theta in shape (2b, 2b), range [0, pi]; phi range [0, 2 * pi]
        theta, phi = S2.meshgrid(b=b, grid_type=grid_type)
        theta = torch.from_numpy(theta)
        phi = torch.from_numpy(phi)

        # x will be reshaped to have one dimension of 1, then can broadcast
        # look this link for more information: https://pytorch.org/docs/stable/notes/broadcasting.html
        x_ = radius * torch.sin(theta) * torch.cos(phi)
        x = x_.reshape((1, 4 * b * b))  # tensor -> [1, 4 * b * b]
        x = x + center[0]

        y_ = radius * torch.sin(theta) * torch.sin(phi)
        y = y_.reshape((1, 4 * b * b))
        y = y + center[1]

        z_ = radius * torch.cos(theta)
        z = z_.reshape((1, 4 * b * b))
        z = z + center[2]

        grid = torch.cat((x, y, z), dim=0)  # -> [3, 4b^2]
        grid = grid.transpose(0, 1)  # -> [4b^2, 3]

        grid = grid.view(2 * b, 2 * b, 3)  # -> [2b, 2b, 3]
        grid = grid.float().cuda()

        grids.append( (radius, grid) )

    assert len(grids) == num_grids
    return grids


def get_cube_grids(b, num_grids, base_radius):
    # Create sphere centers
    x_ = np.linspace(-2/3, 2/3, 3)
    y_ = np.linspace(-2/3, 2/3, 3)
    z_ = np.linspace(-2/3, 2/3, 3)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    
    centers = np.array(list(zip(x, y, z)))
    
    # Create grids based on each center
    all_grids = []
    R = 2 * base_radius * 1/3 * np.sqrt(3) * 1/2
    for center in centers:
        grids = [grid for radius, grid in get_grids(b, num_grids, base_radius=R, center=center)]
        grids = torch.stack(grids, dim=0) # tensor -> [num_grids, 2b, 2b, 3]
        all_grids.append(grids)
    all_grids = torch.stack(all_grids, dim=0) # tensor -> [num_centers, num_grids, 2b, 2b, 3]
    return all_grids

In [183]:
grids = get_cube_grids(b=5, num_grids=3, base_radius=1)
print(grids.shape)
grids = grids.transpose(0, 1)
print(grids.shape)

torch.Size([27, 3, 10, 10, 3])
torch.Size([3, 27, 10, 10, 3])


In [196]:
colors = ["#7b0001", "#ff0001", "#ff8db4"]
fig = ipv.figure()
for i, layer in enumerate(grids):
    layer = layer.reshape(-1, 3).transpose(0, 1)
    x_axis = layer[0, :].cpu().numpy()
    y_axis = layer[1, :].cpu().numpy()
    z_axis = layer[2, :].cpu().numpy()
    ipv.scatter(x_axis, y_axis, z_axis, marker="sphere", color=colors[i])
ipv.show()
ipv.save("./imgs/grid/cube_grids.html")

ValueError: array should be of shape (...,3) or (...,4), not (2700,)

In [187]:
base_radius = 1
num_grids = 3
radiuses = [round(i, 2) for i in list(np.linspace(0, base_radius, num_grids + 1))[1:]]

In [188]:
radiuses

[0.33, 0.67, 1.0]